In [21]:
from vnstock import *
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import ffn
import vnstock
import bt
from backtesting import Backtest, Strategy
from backtesting.test import SMA
from backtesting.lib import crossover
from backtesting.lib import resample_apply
%matplotlib inline

In [44]:
# Define parameters for stock screening insights
params = {"exchangeName": "HOSE,HNX,UPCOM"}

# Get stock screening insights data
df = stock_screening_insights(params, size=1700, drop_lang='vi')

# Merge with listing companies data
listing_df = listing_companies()
df = df.merge(listing_df, how='left', on='ticker')

# Filter for stocks in VN30 index
df = df[df['VN100']==True]

# Filter for stocks meeting conditions: Price above SMA(50), Price above SMA(100), rsi14 between 30 and 75
df = df[(df['priceVsSma50.en'] == 'Price above SMA(50)') & (df['priceVsSMA100.en'] == 'Price above SMA(100)') & (df['priceVsSma50.en'] == 'Price above SMA(50)') & (df['priceVsSMA100.en'] == 'Price above SMA(100)')]

df = df[(df['pe'] < 20) & (df['rsi14'] >= 30) & (df['rsi14'] <= 70) &(df['stockRating'] >= 3)]

# Display the filtered DataFrame
df

,ticker,companyName,marketCap,roe,stockRating,businessOperation,businessModel,financialHealth,alpha,beta,...,VNCOND,VNCONS,VNENE,VNFIN,VNHEAL,VNIND,VNIT,VNMAT,VNREAL,VNUTI
12,ACB,,104287.0,24.8,3.0,4.3,3.6,3.6,0.1,0.8,...,False,False,False,True,False,False,False,False,False,False
120,BMP,,9005.0,39.2,3.1,3.8,3.0,5.0,0.2,0.9,...,False,False,False,False,False,True,False,False,False,False
322,DGC,,45194.0,28.2,3.3,4.5,4.2,4.8,0.4,1.4,...,False,False,False,False,False,False,False,True,False,False
327,DHC,,3542.0,17.2,3.0,3.3,3.5,4.4,0.0,0.8,...,False,False,False,False,False,False,False,True,False,False
502,GMD,,23554.0,28.7,3.2,3.8,4.4,4.6,0.2,1.1,...,False,False,False,False,False,True,False,False,False,False
801,MBB,,122660.0,24.5,3.2,4.7,4.5,3.4,0.1,0.9,...,False,False,False,True,False,False,False,False,False,False
986,PHR,,8740.0,17.9,3.1,2.8,3.6,4.8,0.1,1.1,...,False,False,False,False,False,False,False,True,False,False
1076,PVT,,9224.0,14.5,3.2,2.6,3.0,4.8,0.1,1.4,...,False,False,True,False,False,False,False,False,False,False
1296,TCB,,141957.0,14.8,3.0,4.3,3.4,3.6,0.1,1.1,...,False,False,False,True,False,False,False,False,False,False
1298,TCH,,9455.0,6.5,3.6,3.9,3.0,5.0,0.1,1.6,...,False,False,False,False,False,True,False,False,False,False


In [22]:
def SMA(array, n):
    """Simple moving average"""
    return pd.Series(array).rolling(n).mean()

def RSI(array, n):
    """Relative strength index"""
    # Approximate; good enough
    gain = pd.Series(array).diff()
    loss = gain.copy()
    gain[gain < 0] = 0
    loss[loss > 0] = 0
    rs = gain.ewm(n).mean() / loss.abs().ewm(n).mean()
    return 100 - 100 / (1 + rs)

The strategy roughly goes like this:

Buy a position when:
* daily RSI(30) $>$ 70
* Close $>$ MA(10) $>$ MA(50) $>$ MA(100)
* Volume > MA(20)

Close the position when:
* Daily close is more than 2% _below_ MA(10)
* 8% fixed stop loss is hit

We need to provide bars data in the _lowest time frame_ (i.e. daily) and resample it to any higher time frame (i.e. weekly) that our strategy requires.

In [23]:
class System(Strategy):
    d_rsi = 14  # Daily RSI lookback periods
    w_rsi = 14  # Weekly
    level = 70
    
    def init(self):
        # Compute moving averages the strategy demands
        self.ma10 = self.I(SMA, self.data.Close, 10)
        self.ma50 = self.I(SMA, self.data.Close, 50)
        self.ma100 = self.I(SMA, self.data.Close, 100)
        self.vma20 = self.I(SMA, self.data.Volume, 20)
        # Compute daily RSI(14)
        self.daily_rsi = self.I(RSI, self.data.Close, self.d_rsi)
        
        # To construct weekly RSI, we can use `resample_apply()`
        # helper function from the library
        self.weekly_rsi = resample_apply(
            'W-FRI', RSI, self.data.Close, self.w_rsi)
        
        
    def next(self):
        price = self.data.Close[-1]
        volume = self.data.Volume[-1]
        # If we don't already have a position, and
        # if all conditions are satisfied, enter long.
        if (not self.position and 
            volume > self.vma20[-1] and
            self.daily_rsi[-1] > self.level and
            self.ma10[-1] > self.ma50[-1] > self.ma100[-1] and
            price > self.ma10[-1]):
            
            # Buy at market price on next open, but do
            # set 3% fixed stop loss.
            self.buy(sl=.95 * price)
        
        # If the price closes 2% or more below 10-day MA
        # close the position, if any.
        elif price < .95 * self.ma10[-1]:
            self.position.close()

In [50]:
def process_csv(file_path):
    df = pd.read_csv(file_path)
    df['Date'] = pd.to_datetime(df['Date'])
    df.set_index('Date', inplace=True)
    df = df[~df.index.duplicated(keep='first')]
    return df

csv_files = [('ACB.csv', 'df_acb'), ('BMP.csv', 'df_bmp'), ('DGC.csv', 'df_dgc'),
             ('DHC.csv', 'df_dhc'), ('GMD.csv', 'df_gmd'), ('MBB.csv', 'df_mbb'), 
             ('PHR.csv', 'df_phr'),('PVT.csv', 'df_pvt'), ('TCB.csv', 'df_tcb'), 
             ('TCH.csv', 'df_tch'), ('VIX.csv', 'df_vix'), ('VRE.csv', 'df_vre'),('VSH.csv', 'df_vsh')]

for file, var_name in csv_files:
    df = process_csv(file)
    exec(f"{var_name} = df")

In [55]:
df_dhc.dropna()

,Open,High,Low,Close,Volume
Date,,,,,
2020-05-04,21.51,21.87,21.12,21.31,289090.0
2020-05-05,21.51,21.51,21.17,21.46,87930.0
2020-05-06,21.60,21.63,21.17,21.35,261550.0
2020-05-07,21.46,22.03,21.37,21.69,567310.0
2020-05-08,21.87,21.87,21.35,21.35,304240.0
...,...,...,...,...,...
2024-03-15,46.05,46.10,45.45,45.90,630100.0
2024-03-18,45.90,46.15,43.10,43.70,646600.0
2024-03-19,43.75,44.70,43.75,44.00,306800.0


# ACB

In [25]:
backtest_acb = Backtest(df_acb, System, commission=.002)
acb = backtest_acb.run()
backtest_acb.plot()
acb

C:\Users\Lenovo\anaconda3\Lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
C:\Users\Lenovo\anaconda3\Lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],


Start                     2020-05-04 00:00:00
End                       2024-03-21 00:00:00
Duration                   1417 days 00:00:00
Exposure Time [%]                   34.262126
Equity Final [$]                 16076.242324
Equity Peak [$]                  16250.242324
Return [%]                          60.762423
Buy & Hold Return [%]              235.375852
Return (Ann.) [%]                   13.141192
Volatility (Ann.) [%]               17.867145
Sharpe Ratio                         0.735495
Sortino Ratio                        1.257301
Calmar Ratio                         0.640684
Max. Drawdown [%]                  -20.511187
Avg. Drawdown [%]                   -5.107624
Max. Drawdown Duration      945 days 00:00:00
Avg. Drawdown Duration       89 days 00:00:00
# Trades                                    4
Win Rate [%]                             75.0
Best Trade [%]                      25.657775
Worst Trade [%]                     -5.378853
Avg. Trade [%]                    

In [26]:
%%time
backtest_acb.plot()
backtest_acb.optimize(d_rsi=range(10, 14, 5),
                  w_rsi=range(10, 14, 5),
                  level=range(30, 70, 10))


C:\Users\Lenovo\anaconda3\Lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
C:\Users\Lenovo\anaconda3\Lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],


  0%|          | 0/4 [00:00<?, ?it/s]

CPU times: total: 2.5 s
Wall time: 2.58 s


Start                     2020-05-04 00:00:00
End                       2024-03-21 00:00:00
Duration                   1417 days 00:00:00
Exposure Time [%]                   45.098039
Equity Final [$]                 16484.561017
Equity Peak [$]                  18830.513683
Return [%]                           64.84561
Buy & Hold Return [%]              235.375852
Return (Ann.) [%]                     13.8816
Volatility (Ann.) [%]               20.010303
Sharpe Ratio                         0.693723
Sortino Ratio                        1.185947
Calmar Ratio                         0.404531
Max. Drawdown [%]                  -34.315326
Avg. Drawdown [%]                   -5.037992
Max. Drawdown Duration      990 days 00:00:00
Avg. Drawdown Duration       69 days 00:00:00
# Trades                                    8
Win Rate [%]                             37.5
Best Trade [%]                      32.485919
Worst Trade [%]                     -6.006968
Avg. Trade [%]                    

# DGC

In [48]:
backtest_dgc = Backtest(df_dgc, System, commission=.002)
dgc = backtest_dgc.run()
backtest_dgc.plot()
dgc

C:\Users\Lenovo\anaconda3\Lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
C:\Users\Lenovo\anaconda3\Lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],


Start                     2020-05-04 00:00:00
End                       2024-03-21 00:00:00
Duration                   1417 days 00:00:00
Exposure Time [%]                   34.710744
Equity Final [$]                  18685.20008
Equity Peak [$]                   19428.63336
Return [%]                          86.852001
Buy & Hold Return [%]             1378.365385
Return (Ann.) [%]                   17.673634
Volatility (Ann.) [%]               29.432166
Sharpe Ratio                         0.600487
Sortino Ratio                        1.124916
Calmar Ratio                         0.645026
Max. Drawdown [%]                  -27.399893
Avg. Drawdown [%]                   -7.342266
Max. Drawdown Duration      829 days 00:00:00
Avg. Drawdown Duration       69 days 00:00:00
# Trades                                   13
Win Rate [%]                        46.153846
Best Trade [%]                      51.803585
Worst Trade [%]                     -8.201016
Avg. Trade [%]                    

# DHC

In [57]:
backtest_dhc = Backtest(df_dhc, System, commission=.002)
dhc = backtest_dhc.run()
backtest_dhc.plot()
dhc

ValueError: Some OHLC values are missing (NaN). Please strip those lines with `df.dropna()` or fill them in with `df.interpolate()` or whatever.

In [14]:
backtest2.plot()

C:\Users\Lenovo\anaconda3\Lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
C:\Users\Lenovo\anaconda3\Lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],


GridPlot(id='p2843', ...)

In [15]:
%%time

backtest2.optimize(d_rsi=range(10, 14, 5),
                  w_rsi=range(10, 14, 5),
                  level=range(30, 70, 10))

  0%|          | 0/4 [00:00<?, ?it/s]

CPU times: total: 812 ms
Wall time: 836 ms


Start                     2020-05-04 00:00:00
End                       2024-03-19 00:00:00
Duration                   1415 days 00:00:00
Exposure Time [%]                   46.604938
Equity Final [$]                   16874.2824
Equity Peak [$]                   19299.75064
Return [%]                          68.742824
Buy & Hold Return [%]              106.475833
Return (Ann.) [%]                   14.527628
Volatility (Ann.) [%]                21.96916
Sharpe Ratio                         0.661274
Sortino Ratio                        1.202762
Calmar Ratio                          0.65654
Max. Drawdown [%]                  -22.127556
Avg. Drawdown [%]                   -4.851002
Max. Drawdown Duration      821 days 00:00:00
Avg. Drawdown Duration       68 days 00:00:00
# Trades                                    7
Win Rate [%]                        57.142857
Best Trade [%]                      32.729932
Worst Trade [%]                     -5.619697
Avg. Trade [%]                    

In [16]:
backtest2.plot()

C:\Users\Lenovo\anaconda3\Lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
C:\Users\Lenovo\anaconda3\Lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],


GridPlot(id='p3322', ...)

# GMD

In [18]:
backtest3 = Backtest(df3, System, commission=.002)
backtest3.run()

Start                     2020-05-04 00:00:00
End                       2024-03-19 00:00:00
Duration                   1415 days 00:00:00
Exposure Time [%]                   17.078189
Equity Final [$]                    8464.7238
Equity Peak [$]                   12818.05684
Return [%]                         -15.352762
Buy & Hold Return [%]               384.88665
Return (Ann.) [%]                   -4.229237
Volatility (Ann.) [%]               14.263856
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -34.427822
Avg. Drawdown [%]                  -11.393357
Max. Drawdown Duration     1162 days 00:00:00
Avg. Drawdown Duration      251 days 00:00:00
# Trades                                    9
Win Rate [%]                        22.222222
Best Trade [%]                      20.903387
Worst Trade [%]                      -6.32702
Avg. Trade [%]                    

In [19]:
%%time

backtest3.optimize(d_rsi=range(10, 14, 5),
                  w_rsi=range(10, 14, 5),
                  level=range(30, 70, 10))

  0%|          | 0/4 [00:00<?, ?it/s]

CPU times: total: 875 ms
Wall time: 824 ms


Start                     2020-05-04 00:00:00
End                       2024-03-19 00:00:00
Duration                   1415 days 00:00:00
Exposure Time [%]                   50.720165
Equity Final [$]                  10352.19362
Equity Peak [$]                   14965.18298
Return [%]                           3.521936
Buy & Hold Return [%]               384.88665
Return (Ann.) [%]                    0.901422
Volatility (Ann.) [%]               23.510257
Sharpe Ratio                         0.038342
Sortino Ratio                        0.056726
Calmar Ratio                          0.02261
Max. Drawdown [%]                  -39.868469
Avg. Drawdown [%]                   -9.338105
Max. Drawdown Duration      930 days 00:00:00
Avg. Drawdown Duration      115 days 00:00:00
# Trades                                   16
Win Rate [%]                            31.25
Best Trade [%]                      27.417579
Worst Trade [%]                     -5.876924
Avg. Trade [%]                    

In [20]:
backtest3.plot()

C:\Users\Lenovo\anaconda3\Lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
C:\Users\Lenovo\anaconda3\Lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],


GridPlot(id='p3737', ...)

In [58]:
backtest_vre = Backtest(df_vre, System, commission=.002)
vre = backtest_vre.run()
backtest_vre.plot()
vre

C:\Users\Lenovo\anaconda3\Lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
C:\Users\Lenovo\anaconda3\Lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],


Start                     2020-05-04 00:00:00
End                       2024-03-21 00:00:00
Duration                   1417 days 00:00:00
Exposure Time [%]                    3.798768
Equity Final [$]                    9404.6985
Equity Peak [$]                     12488.068
Return [%]                          -5.953015
Buy & Hold Return [%]               14.925373
Return (Ann.) [%]                   -1.575413
Volatility (Ann.) [%]                8.355389
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -24.690525
Avg. Drawdown [%]                   -4.450397
Max. Drawdown Duration     1151 days 00:00:00
Avg. Drawdown Duration      149 days 00:00:00
# Trades                                    3
Win Rate [%]                        33.333333
Best Trade [%]                       5.910695
Worst Trade [%]                      -6.11267
Avg. Trade [%]                    

In [59]:
%%time
backtest_vre.plot()
backtest_vre.optimize(d_rsi=range(10, 14, 5),
                  w_rsi=range(10, 14, 5),
                  level=range(30, 70, 10))


C:\Users\Lenovo\anaconda3\Lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
C:\Users\Lenovo\anaconda3\Lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],


  0%|          | 0/4 [00:00<?, ?it/s]

CPU times: total: 2.97 s
Wall time: 3.11 s


Start                     2020-05-04 00:00:00
End                       2024-03-21 00:00:00
Duration                   1417 days 00:00:00
Exposure Time [%]                   11.498973
Equity Final [$]                    7192.3722
Equity Peak [$]                    12679.5916
Return [%]                         -28.076278
Buy & Hold Return [%]               14.925373
Return (Ann.) [%]                     -8.1733
Volatility (Ann.) [%]               12.401405
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -43.275995
Avg. Drawdown [%]                   -7.566096
Max. Drawdown Duration     1151 days 00:00:00
Avg. Drawdown Duration      159 days 00:00:00
# Trades                                   10
Win Rate [%]                             20.0
Best Trade [%]                      13.422092
Worst Trade [%]                     -8.349967
Avg. Trade [%]                    

In [62]:
backtest_vix = Backtest(df_vix, System, commission=.002)
vix = backtest_vix.run()
backtest_vix.plot()
vix

C:\Users\Lenovo\anaconda3\Lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
C:\Users\Lenovo\anaconda3\Lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],


Start                     2020-05-04 00:00:00
End                       2024-03-21 00:00:00
Duration                   1417 days 00:00:00
Exposure Time [%]                   18.821096
Equity Final [$]                   25156.1003
Equity Peak [$]                    27729.2795
Return [%]                         151.561003
Buy & Hold Return [%]             1064.739884
Return (Ann.) [%]                   27.176704
Volatility (Ann.) [%]               34.286013
Sharpe Ratio                         0.792647
Sortino Ratio                        1.687617
Calmar Ratio                         1.280013
Max. Drawdown [%]                  -21.231586
Avg. Drawdown [%]                   -7.983972
Max. Drawdown Duration      544 days 00:00:00
Avg. Drawdown Duration       69 days 00:00:00
# Trades                                   13
Win Rate [%]                        46.153846
Best Trade [%]                      37.924932
Worst Trade [%]                     -6.953535
Avg. Trade [%]                    

In [63]:
%%time
backtest_vix.plot()
backtest_vix.optimize(d_rsi=range(10, 14, 5),
                  w_rsi=range(10, 14, 5),
                  level=range(30, 70, 10))


C:\Users\Lenovo\anaconda3\Lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
C:\Users\Lenovo\anaconda3\Lib\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],


  0%|          | 0/4 [00:00<?, ?it/s]

CPU times: total: 2.55 s
Wall time: 2.64 s


Start                     2020-05-04 00:00:00
End                       2024-03-21 00:00:00
Duration                   1417 days 00:00:00
Exposure Time [%]                   28.335057
Equity Final [$]                    67281.765
Equity Peak [$]                     68100.265
Return [%]                          572.81765
Buy & Hold Return [%]             1064.739884
Return (Ann.) [%]                   64.342498
Volatility (Ann.) [%]               50.663427
Sharpe Ratio                         1.269999
Sortino Ratio                        3.853889
Calmar Ratio                         2.596657
Max. Drawdown [%]                  -24.778977
Avg. Drawdown [%]                   -7.165942
Max. Drawdown Duration      548 days 00:00:00
Avg. Drawdown Duration       44 days 00:00:00
# Trades                                   11
Win Rate [%]                        54.545455
Best Trade [%]                      78.620033
Worst Trade [%]                     -6.196191
Avg. Trade [%]                    